In [ ]:
#!pip install opencv-python
import numpy as np
import cv2

In [ ]:
TILE_SIZE = 32
OFS = 50

In [ ]:
MARKET = """
##################
##....c...c......#
##..DE..ES..SB..A#
##c.DEc.ES..SAc.B#
##..DE..ES.cSB..A#
##.cDE..ES..SA..B#
##..DE..ESc.SB..A#
##..c.........c..#
##..CC..CC..CC...#
##..c..........c.#
##...........c...#
##############GG##
""".strip()

In [ ]:
class SupermarketMap:
    """Visualizes the supermarket background"""

    def __init__(self, layout, tiles):
        """
        layout : a string with each character representing a tile
        tile   : a numpy array containing the tile image
        """
        self.tiles = tiles
        self.contents = [list(row) for row in layout.split("\n")]
        self.xsize = len(self.contents[0])
        self.ysize = len(self.contents)
        self.image = np.zeros(
            (self.ysize * TILE_SIZE, self.xsize * TILE_SIZE, 3), dtype=np.uint8
        )
        self.prepare_map()

    def get_tile(self, char):
        """returns the array for a given tile character"""
        if char == "#":
            return self.tiles[0:32, 0:32]
        elif char == "G": # gates
            return self.tiles[7 * 32 : 8 * 32, 3 * 32 : 4 * 32] # 7th row, 3rd column
        elif char == "C": # checkout
            return self.tiles[2 * 32 : 3 * 32, 8 * 32 : 9 * 32]
        elif char == "A": # apple --> fruits
            return self.tiles[1 * 32 : 2 * 32, 4 * 32 : 5 * 32] 
        elif char == "B": # banana
            return self.tiles[0 * 32 : 1 * 32, 4 * 32 : 5 * 32] # 0th row, 4st column (tile)
        elif char == "S": # spices
            return self.tiles[1 * 32 : 2 * 32, 3 * 32 : 4 * 32] # 1st row, 3st column (tile)
        elif char == "E": # eggs --> dairy
            return self.tiles[7 * 32 : 8 * 32, 11 * 32 : 12 * 32] # 7th row, 11st column (tile)
        elif char == "D": # drinks
            return self.tiles[6 * 32 : 7 * 32, 13 * 32 : 14 * 32] # 6th row, 13st column (tile)
        elif char == "c": # consumers
            return self.tiles[3 * 32 : 4 * 32, 0 * 32 : 1 * 32] # 3rd row, 0 column (tile)
        else:
            return self.tiles[32:64, 64:96]

    def prepare_map(self):
        """prepares the entire image as a big numpy array"""
        for y, row in enumerate(self.contents):
            for x, tile in enumerate(row):
                bm = self.get_tile(tile)
                self.image[
                    y * TILE_SIZE : (y + 1) * TILE_SIZE,
                    x * TILE_SIZE : (x + 1) * TILE_SIZE,
                ] = bm

    def draw(self, frame, offset=OFS):
        """
        draws the image into a frame
        offset pixels from the top left corner
        """
        frame[
            OFS : OFS + self.image.shape[0], OFS : OFS + self.image.shape[1]
        ] = self.image

    def write_image(self, filename):
        """writes the image into a file"""
        cv2.imwrite(filename, self.image)

if __name__ == "__main__":

    background = np.zeros((700, 1000, 3), np.uint8)
    tiles = cv2.imread("tiles.png") # the cv2.imread() function reads in the image as a NumPy array, so it can be sliced within the program.

    market = SupermarketMap(MARKET, tiles)


    while True:
        frame = background.copy()
        market.draw(frame)

        cv2.imshow("frame", frame) # the cv2.imshow() method is what’s actually displaying each frame on the screen

        key = chr(cv2.waitKey(1) & 0xFF)
        if key == "q":
            break

    cv2.destroyAllWindows()
    
    market.write_image("supermarket.png")